In [11]:

import os
import numpy as np
import pandas as pd
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from matplotlib import pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation
import warnings
warnings.filterwarnings('ignore')
import spacy,re
import gensim

In [5]:
# Reading data via pandas lib.


review = pd.read_excel(r"D:\Manipal Data Science\Term 3\Practice\Amazon.xlsx")


reviews = review.rename({'reviews.text':'text'},axis=1)
reviews.head()

,id,asins,brand,categories,colors,dateAdded,dateUpdated,dimension,ean,keys,...,reviews.rating,reviews.sourceURLs,text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sizes,upc,weight
0,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I initially had trouble deciding between the p...,"Paperwhite voyage, no regrets!",NaN,NaN,Cristina M,NaN,NaN,205 grams
1,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,Allow me to preface this with a little history...,One Simply Could Not Ask For More,NaN,NaN,Ricky,NaN,NaN,205 grams
2,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,4.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I am enjoying it so far. Great for reading. Ha...,Great for those that just want an e-reader,NaN,NaN,Tedd Gardiner,NaN,NaN,205 grams
3,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I bought one of the first Paperwhites and have...,Love / Hate relationship,NaN,NaN,Dougal,NaN,NaN,205 grams
4,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I have to say upfront - I don't like coroporat...,I LOVE IT,NaN,NaN,Miljan David Tanic,NaN,NaN,205 grams


#### Remove emails and newline characters

In [9]:
# Convert to list
data = reviews.text.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

print(data[:1])

['I initially had trouble deciding between the paperwhite and the voyage because reviews more or less said the same thing: the paperwhite is great, but if you have spending money, go for the voyage.Fortunately, I had friends who owned each, so I ended up buying the paperwhite on this basis: both models now have 300 ppi, so the 80 dollar jump turns out pricey the voyages page press isnt always sensitive, and if you are fine with a specific setting, you dont need auto light adjustment).Its been a week and I am loving my paperwhite, no regrets! The touch screen is receptive and easy to use, and I keep the light at a specific setting regardless of the time of day. (In any case, its not hard to change the setting either, as youll only be changing the light level at a certain time of day, not every now and then while reading).Also glad that I went for the international shipping option with Amazon. Extra expense, but delivery was on time, with tracking, and I didnt need to worry about customs

#### Tokenize and Clean-up using gensim’s simple_preprocess()

In [12]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['initially', 'had', 'trouble', 'deciding', 'between', 'the', 'paperwhite', 'and', 'the', 'voyage', 'because', 'reviews', 'more', 'or', 'less', 'said', 'the', 'same', 'thing', 'the', 'paperwhite', 'is', 'great', 'but', 'if', 'you', 'have', 'spending', 'money', 'go', 'for', 'the', 'voyage', 'fortunately', 'had', 'friends', 'who', 'owned', 'each', 'so', 'ended', 'up', 'buying', 'the', 'paperwhite', 'on', 'this', 'basis', 'both', 'models', 'now', 'have', 'ppi', 'so', 'the', 'dollar', 'jump', 'turns', 'out', 'pricey', 'the', 'voyages', 'page', 'press', 'isnt', 'always', 'sensitive', 'and', 'if', 'you', 'are', 'fine', 'with', 'specific', 'setting', 'you', 'dont', 'need', 'auto', 'light', 'adjustment', 'its', 'been', 'week', 'and', 'am', 'loving', 'my', 'paperwhite', 'no', 'regrets', 'the', 'touch', 'screen', 'is', 'receptive', 'and', 'easy', 'to', 'use', 'and', 'keep', 'the', 'light', 'at', 'specific', 'setting', 'regardless', 'of', 'the', 'time', 'of', 'day', 'in', 'any', 'case', 'its', '

#### convert words to its root word via Lemmatization

In [13]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['initially trouble decide voyage review more less say same thing paperwhite great spend money go voyage fortunately friend own so end buy paperwhite basis model now so dollar jump turn voyage page press always sensitive fine specific setting need auto light adjustment week love paperwhite regret touch screen receptive easy use keep light specific set regardless time day case hard change setting only change light level certain time day now then read also glad go international shipping option extra expense delivery time tracking need worry custom use third party shipping service', 'allow preface little history casual reader own nook simple touch have read girl dragon brave new world few other key title fair say nook as much use many other get fast forward today have full week new kindle paperwhite admit be love just kindle read all over again now let relate review love read back kindle investment experience receive when buy kindle simply pay screen entire experience include buy read hun

### creating the documents term matrix

In [14]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum read occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}')  # num chars > 3 

data_vectorized = vectorizer.fit_transform(data_lemmatized)

#### Build LDA model 

In [16]:
lda_model = LatentDirichletAllocation(n_components=10,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=100,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=100, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [23]:
from sklearn.model_selection import GridSearchCV


# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                   

In [25]:
# Best Model
best_lda_model = model.best_estimator_

In [33]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

,ability,able,absolutely,access,accessible,accidental,accord,account,accuracy,accurate,...,word,work,workaround,world,worry,worth,write,wrong,year,youtube
Topic0,0.100000,46.052145,0.100002,0.100001,0.100004,0.100000,0.100000,0.100001,0.100000,0.100002,...,0.100002,44.522553,0.1,0.100001,0.100001,0.100001,0.100000,0.100005,0.100000,0.100001
Topic1,0.100006,63.319428,10.377118,0.100003,15.216946,0.100000,0.100006,0.100002,28.099961,82.099576,...,0.100007,0.100003,0.1,0.100001,0.100003,0.100004,126.222063,0.100004,219.207516,28.008527
Topic2,0.875026,6.676675,0.100007,53.316560,0.100010,0.100001,0.100010,0.100075,0.100009,0.100002,...,17.399449,51.686173,0.1,0.100027,0.100007,16.536870,0.100014,6.869699,0.404958,6.091745
Topic3,17.030582,52.798013,9.744277,25.968279,0.100022,0.100000,0.100003,16.527801,0.100001,0.100013,...,34.991709,149.769888,0.1,24.057598,3.808751,11.221564,16.157679,13.629224,6.094917,0.100024
Topic4,0.100007,5.401943,0.251059,43.032650,0.100001,0.100000,0.100000,0.100005,0.100000,0.100006,...,0.100009,154.433601,0.1,0.100003,0.100001,0.100002,0.100006,14.815860,9.759548,13.746284


In [27]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics


,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Doc0,0,0.05,0,0.28,0,0,0,0.23,0,0.42,9
Doc1,0,0,0.06,0.37,0,0.11,0.09,0.28,0.03,0.06,3
Doc2,0.01,0.01,0.13,0.35,0.01,0.01,0.01,0.47,0.01,0.01,7
Doc3,0.05,0,0.07,0.21,0.07,0.17,0,0.28,0.06,0.09,7
Doc4,0,0.03,0.05,0.33,0,0,0,0.22,0,0.36,9
Doc5,0,0,0,0.25,0.08,0,0.48,0,0.17,0,6
Doc6,0,0,0.06,0.37,0,0.11,0.09,0.28,0.03,0.06,3
Doc7,0,0.05,0,0.1,0,0,0,0.4,0,0.43,9
Doc8,0,0.05,0,0.28,0,0,0,0.23,0,0.42,9
Doc9,0.01,0.01,0.13,0.35,0.01,0.01,0.01,0.47,0.01,0.01,7


#### Review topics distribution across documents

In [29]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,3,617
1,9,270
2,5,133
3,7,115
4,1,92
5,4,87
6,2,84
7,6,79
8,8,68
9,0,52


## Show top n keywords for each topic

In [36]:

def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,headphone,ear,earbud,design,magnet,use,just,really,sound,noise,suppose,lot,flat,simple,hate
Topic 1,sound,apple,feel,headphone,bud,year,people,review,compare,range,fall,use,hdx,read,product
Topic 2,tablet,want,screen,great,use,price,camera,large,read,quality,kindle,nice,easy,low,high
Topic 3,tap,great,use,echo,love,speaker,sound,buy,alexa,good,music,just,easy,work,read
Topic 4,headphone,work,comcast,look,set,nice,sound,volume,think,love,thing,free,just,want,earpod
Topic 5,device,prime,use,search,content,year,movie,game,app,available,video,review,watch,play,work
Topic 6,prime,video,kindle,device,download,time,title,watch,compare,feature,year,display,new,book,read
Topic 7,kindle,read,book,paperwhite,new,model,reader,screen,review,display,touch,year,early,keyboard,old
Topic 8,roku,box,use,content,feature,option,apple,product,review,read,item,smartphone,device,include,stream
Topic 9,case,cover,look,work,time,use,just,make,remote,dot,purchase,new,say,screen,battery


### visualize the LDA model with pyLDAvis

In [32]:
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()


panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3      154.784698  -62.480610       1        1  13.783525
5       73.847282 -160.525589       2        1  12.600109
9       17.109896   92.322014       3        1  12.188567
1      -38.428089 -137.195679       4        1  11.036523
7      -61.714493  -10.086044       5        1  10.524568
8      -95.172455  115.633614       6        1   9.760834
2     -176.109070   17.587770       7        1   8.453313
6      127.628258   61.722050       8        1   7.827657
4       40.394295  -34.794395       9        1   7.822495
0     -148.948135 -106.612915      10        1   6.002408, topic_info=     Category         Freq         Term        Total  loglift  logprob
436   Default   818.000000    headphone   818.000000  30.0000  30.0000
516   Default  1310.000000       kindle  1310.000000  29.0000  29.0000
840   Default   367.000000         roku   367.000000  28.0000  28.0000
750   Default   700.000000        prime   700.000000  27.0000  27.0000
286   Default   415.000000          ear   415.000000  26.0000  26.0000
981   Default   806.000000       tablet   806.000000  25.0000  25.0000
287   Default   258.000000       earbud   258.000000  24.0000  24.0000
920   Default   892.000000        sound   892.000000  23.0000  23.0000
986   Default   381.000000          tap   381.000000  22.0000  22.0000
106   Default   344.000000          box   344.000000  21.0000  21.0000
251   Default   258.000000       design   258.000000  20.0000  20.0000
207   Default   590.000000      content   590.000000  19.0000  19.0000
574   Default   222.000000       magnet   222.000000  18.0000  18.0000
856   Default   685.000000       screen   685.000000  17.0000  17.0000
1068  Default   709.000000         want   709.000000  16.0000  16.0000
410   Default   795.000000        great   795.000000  15.0000  15.0000
1060  Default   440.000000        video   440.000000  14.0000  14.0000
687   Default   277.000000   paperwhite   277.000000  13.0000  13.0000
45    Default   493.000000        apple   493.000000  12.0000  12.0000
296   Default   355.000000         echo   355.000000  11.0000  11.0000
1095  Default   736.000000         year   736.000000  10.0000  10.0000
102   Default   539.000000         book   539.000000   9.0000   9.0000
119   Default   285.000000          bud   285.000000   8.0000   8.0000
252   Default   990.000000       device   990.000000   7.0000   7.0000
694   Default   375.000000       people   375.000000   6.0000   6.0000
28    Default   239.000000        alexa   239.000000   5.0000   5.0000
924   Default   346.000000      speaker   346.000000   4.0000   4.0000
789   Default   223.000000       reader   223.000000   3.0000   3.0000
273   Default   331.000000     download   331.000000   2.0000   2.0000
629   Default   634.000000          new   634.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
424   Topic10    47.259751         hang    55.507200   2.6522  -4.8063
94    Topic10    45.020746        block    54.768563   2.6170  -4.8548
634   Topic10   130.397588        noise   163.158661   2.5889  -3.7913
51    Topic10    45.693152       aspect    59.523984   2.5486  -4.8400
844   Topic10    42.981409         safe    57.572020   2.5207  -4.9012
985   Topic10    85.781973       tangle   117.300332   2.5001  -4.2101
676   Topic10    45.918260      outside    64.992359   2.4656  -4.8351
69    Topic10    44.857023          bag    66.855239   2.4140  -4.8585
365   Topic10    44.695792       finish    67.427349   2.4018  -4.8621
66    Topic10    89.360978      awesome   141.298297   2.3548  -4.1692
286   Topic10   258.141296          ear   415.090709   2.3380  -3.1084
957   Topic10    44.844695        style    72.471515   2.3330  -4.8587
176   Topic10    86.292479  comfortable   142.666802   2.3102  -4.2042
63    Topic10    46.015568      average    78.397465   2.2802  -4.

## Predict the topics for a new piece of text


In [37]:
# Define function to predict topic for a given text document.
nlp = spacy.load('en', disable=['parser', 'ner'])

def predict_topic(text, nlp=nlp):
    global sent_to_words
    global lemmatization

    # Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))

    # Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Step 3: Vectorize transform
    mytext_4 = vectorizer.transform(mytext_3)

    # Step 4: LDA Transform
    topic_probability_scores = best_lda_model.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    return topic, topic_probability_scores

# Predict the topic
mytext = ["Some text about christianity and bible"]
topic, prob_scores = predict_topic(text = mytext)
print(topic)

['tablet', 'want', 'screen', 'great', 'use', 'price', 'camera', 'large', 'read', 'quality', 'kindle', 'nice', 'easy', 'low', 'high']
